In [58]:
import pandas as pd
import panel as pn
import holoviews as hv
import numpy as np
import panel as pn

def interactive_plot(data_frame):
    # (1) Check if DataFrame has at least 3 columns
    if len(data_frame.columns) < 3:
        raise ValueError("The DataFrame should have at least 3 columns. Use a different function for this data.")

    # Reshape DataFrame to multi-dimensional array for easier indexing
    df_shape = [len(data_frame.index)] + [len(level) for level in data_frame.columns.levels]
    data = data_frame.values.reshape(df_shape)

    # (2) & (3) Define plotting functions
    def plot_1D(*indices):
        idx = [slice(None) if i < 2 else indices[i-2] for i in range(len(df_shape))]
        print(f'idx={idx}', flush=True)
        return hv.Curve(data[tuple(idx)])

    def plot_2D(*indices):
        idx = [slice(None) if i < 2 else indices[i-3] for i in range(len(df_shape))]
        return hv.Image(data[tuple(idx)], kdims=['Dim 0', 'Dim 1'])

    # (4) Create widgets
    dim_selector = pn.widgets.RadioBoxGroup(name='Dimension', options=['1D', '2D'], inline=True)

    index_names = data_frame.columns.names
    print(index_names)

    sliders = [pn.widgets.IntSlider(name=index_names[i], start=0, end=df_shape[i]-1) for i in range(len(index_names))]

    special_dim_0 = pn.widgets.IntInput(name='Special Dimension 0', value=0, step=1)
    special_dim_1 = pn.widgets.IntInput(name='Special Dimension 1', value=1, step=1)

    @pn.depends(special_dim_0.param.value, special_dim_1.param.value)
    def update_slider_names(special_dim_0_value, special_dim_1_value):
        for i, slider in enumerate(sliders):
            dim_idx = i + 2  # Adjust index to skip the first two dimensions
            if dim_idx == special_dim_0_value:
                slider.name = f'Special Dim 0: {index_names[dim_idx]}'
            elif dim_idx == special_dim_1_value:
                slider.name = f'Special Dim 1: {index_names[dim_idx]}'
            else:
                slider.name = f'{index_names[dim_idx]}'

    @pn.depends(dim_selector.param.value, *sliders, special_dim_0.param.value, special_dim_1.param.value)
    def reactive_plot(dim, *indices, special_dim_0=0, special_dim_1=1):
        # Update the index positions based on special dimensions
        if dim == '1D':
            indices = indices[:special_dim_0] + (slice(None),) + indices[special_dim_0:special_dim_1] + (slice(None),) + indices[special_dim_1:]
            print(f'indices1D={indices}', flush=True)
            return plot_1D(*indices)
        else:
            indices = indices[:special_dim_0] + (slice(None), slice(None)) + indices[special_dim_0:]
            print(f'indices2D={indices}', flush=True)
            return plot_2D(*indices)

    # (5) Create layout and return
    layout = pn.Row(
        pn.Column(dim_selector, *sliders, special_dim_0, special_dim_1, update_slider_names),
        reactive_plot
    )
    return layout

# Example usage:
# Assume df is your DataFrame
# layout = interactive_plot(df)
# layout.servable()  # or layout.show()


In [59]:
pn.extension()
np.random.seed(0)  # Seed for reproducibility
data_shape = (5, 4, 3, 2)  # Shape of the multi-dimensional data
random_data = np.array(list(range(int(np.prod(data_shape))))).reshape(*data_shape) + np.random.randn(*data_shape) * 5.0  # High variance random data

# Create a DataFrame with a multi-level column index
columns = pd.MultiIndex.from_product([range(dim_size) for dim_size in data_shape[1:]],
                                     names=[f'Dim {i}' for i in range(1,len(data_shape))])
df = pd.DataFrame(random_data.reshape(data_shape[0], -1), columns=columns)

# Create interactive plot layout
layout = interactive_plot(df)

# Display the layout
layout.servable()

['Dim 1', 'Dim 2', 'Dim 3']


IndexError: list index out of range